# Chat Completion with PLLuM-8x7B-chat GGUF

This notebook demonstrates how to implement a chat completion interface with the PLLuM-8x7B-chat model, including handling conversation history.

## Prerequisites

- Download a quantized model from [Hugging Face](https://huggingface.co/piotrmaciejbednarski/PLLuM-8x7B-chat-GGUF)
- Install required packages: `pip install llama-cpp-python`

In [ ]:
# Install required packages if not already installed
%pip install llama-cpp-python

## 1. Load the Model

First, we'll load the model using the `Llama` class.

In [ ]:
from llama_cpp import Llama
import os

# Set path to the model file - update this to your model's location
model_path = "../models/PLLuM-8x7B-chat-gguf-q4_k_m.gguf"

# Check if the model file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}. Please download it first.")

# Load the model
llm = Llama(
    model_path=model_path,
    n_ctx=4096,      # Context window size
    n_threads=8,     # Number of CPU threads to use
    n_batch=512,     # Batch size for prompt processing
    verbose=False    # Set to True for more detailed logs
)

print(f"Model loaded successfully: {model_path}")

## 2. Implementing a Chat Interface

Let's implement a simple chat interface that keeps track of conversation history.

In [ ]:
class ChatBot:
    def __init__(self, llm, system_prompt=None):
        self.llm = llm
        self.conversation_history = []
        
        # Set default system prompt if none provided
        if system_prompt is None:
            self.system_prompt = "Jesteś pomocnym, uprzejmym i dokładnym asystentem AI o nazwie PLLuM. "\
                                "Odpowiadasz na pytania użytkownika w języku polskim w sposób przyjazny i profesjonalny."
        else:
            self.system_prompt = system_prompt
            
        # Add system prompt to history
        self.conversation_history.append({"role": "system", "content": self.system_prompt})
    
    def get_prompt_template(self):
        """Create a formatted prompt from the conversation history."""
        prompt = ""
        
        for message in self.conversation_history:
            role = message["role"]
            content = message["content"]
            
            if role == "system":
                # System messages are typically not shown in the prompt template
                # but can be included at the beginning
                if prompt == "":
                    prompt += f"Instrukcja: {content}\n\n"
            elif role == "user":
                prompt += f"Użytkownik: {content}\n"
            elif role == "assistant":
                prompt += f"Asystent: {content}\n"
        
        # Add the assistant prefix for the response
        prompt += "Asystent: "
        
        return prompt
    
    def chat(self, user_message, max_tokens=512, temperature=0.7):
        """Process a user message and generate a response."""
        # Add the user message to history
        self.conversation_history.append({"role": "user", "content": user_message})
        
        # Create the prompt from conversation history
        prompt = self.get_prompt_template()
        
        # Generate response
        response = self.llm(
            prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95,
            top_k=50,
            repeat_penalty=1.1,
            stop=["\nUżytkownik:", "\nUser:"]  # Stop generating at new user message
        )
        
        assistant_response = response["choices"][0]["text"]
        
        # Add the assistant response to history
        self.conversation_history.append({"role": "assistant", "content": assistant_response})
        
        return assistant_response
    
    def reset_conversation(self, system_prompt=None):
        """Reset the conversation history."""
        if system_prompt is not None:
            self.system_prompt = system_prompt
            
        self.conversation_history = [
            {"role": "system", "content": self.system_prompt}
        ]
        
    def view_conversation_history(self):
        """Display the conversation history."""
        for message in self.conversation_history:
            if message["role"] == "system":
                print(f"[System] {message['content']}")
            elif message["role"] == "user":
                print(f"[User] {message['content']}")
            elif message["role"] == "assistant":
                print(f"[Assistant] {message['content']}")

## 3. Create and Test the Chat Bot

Let's create a chatbot instance and test it with a few messages.

In [ ]:
# Create a chatbot with a custom system prompt
system_prompt = (
    "Jesteś pomocnym, uprzejmym i dokładnym asystentem AI o imieniu PLLuM, "
    "stworzonym przez polskie Ministerstwo Cyfryzacji. "
    "Odpowiadasz na pytania użytkownika w języku polskim, chyba że zostaniesz poproszony o inny język. "
    "Twoje odpowiedzi są zwięzłe, merytoryczne i pomocne. "
    "Jeśli nie znasz odpowiedzi na pytanie, przyznaj to otwarcie."
)

chatbot = ChatBot(llm, system_prompt="".join(system_prompt))

# Test the first message
user_message = "Cześć! Jak się masz?"
print(f"[User] {user_message}")

response = chatbot.chat(user_message)
print(f"[Assistant] {response}")

## 4. Continue the Conversation

Let's continue the conversation with a few more messages to test context retention.

In [ ]:
# Ask a follow-up question
user_message = "Powiedz mi coś o sobie. Kim jesteś?"
print(f"[User] {user_message}")

response = chatbot.chat(user_message)
print(f"[Assistant] {response}")

In [ ]:
# Ask for some information
user_message = "Możesz mi wyjaśnić, co to jest sztuczna inteligencja?"
print(f"[User] {user_message}")

response = chatbot.chat(user_message)
print(f"[Assistant] {response}")

In [ ]:
# Reference the previous conversation
user_message = "Czy możesz podać mi kilka przykładów zastosowań sztucznej inteligencji w codziennym życiu?"
print(f"[User] {user_message}")

response = chatbot.chat(user_message)
print(f"[Assistant] {response}")

## 5. View the Complete Conversation History

In [ ]:
# Display the complete conversation
print("Complete Conversation History:")
print("-" * 50)
chatbot.view_conversation_history()

## 6. Reset and Start a New Conversation

Let's reset the conversation and start a new one with a different system prompt.

In [ ]:
# Reset the conversation with a new system prompt
new_system_prompt = (
    "Jesteś ekspertem w dziedzinie historii Polski. "
    "Odpowiadasz na pytania w sposób rzeczowy, podając istotne fakty, daty i kontekst historyczny. "
    "Twoje odpowiedzi są zwięzłe, ale bogate w informacje."
)

chatbot.reset_conversation(system_prompt=new_system_prompt)
print("Conversation has been reset with a new system prompt.")

# Test the new conversation
user_message = "Opowiedz mi o najważniejszych wydarzeniach w historii Polski XX wieku."
print(f"[User] {user_message}")

response = chatbot.chat(user_message, max_tokens=1024)
print(f"[Assistant] {response}")

## 7. Implementing a Simple Interactive Chat Loop

Let's create a simple interactive chat interface using IPython widgets.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create an interactive chat interface
def interactive_chat():
    # Reset chatbot with general purpose prompt
    general_system_prompt = "Jesteś pomocnym, uprzejmym i dokładnym asystentem AI o nazwie PLLuM. Odpowiadasz na pytania w sposób przyjazny i profesjonalny."
    chatbot.reset_conversation(system_prompt=general_system_prompt)
    
    # Chat history display
    output = widgets.Output()
    
    # Text input for user messages
    text_input = widgets.Text(
        placeholder='Wpisz wiadomość...',
        description='',
        layout=widgets.Layout(width='80%')
    )
    
    # Send button
    send_button = widgets.Button(
        description='Wyślij',
        button_style='primary',
        layout=widgets.Layout(width='18%')
    )
    
    # Reset button
    reset_button = widgets.Button(
        description='Reset',
        button_style='danger',
        layout=widgets.Layout(width='18%')
    )
    
    # Layout for input area
    input_area = widgets.HBox([text_input, send_button, reset_button])
    
    # Display welcome message
    with output:
        print("Welcome to PLLuM Chat! Type a message and press 'Send' or hit Enter.")
        print("-" * 50)
    
    # Message handler
    def on_send(b):
        user_message = text_input.value
        if not user_message.strip():
            return
            
        # Clear the input field
        text_input.value = ''
        
        with output:
            print(f"\n[User] {user_message}")
            print("[Assistant] Thinking...")
            
            # Get response from chatbot
            response = chatbot.chat(user_message)
            
            # Clear the "thinking" message and display the response
            clear_output(wait=True)
            chatbot.view_conversation_history()
    
    # Reset handler
    def on_reset(b):
        chatbot.reset_conversation()
        with output:
            clear_output()
            print("Conversation has been reset.")
            print("-" * 50)
    
    # Handle Enter key
    def on_enter(sender):
        on_send(None)
    
    # Attach event handlers
    send_button.on_click(on_send)
    reset_button.on_click(on_reset)
    text_input.on_submit(on_enter)
    
    # Display the widget
    display(widgets.VBox([output, input_area]))

# Launch the interactive chat
interactive_chat()

## 8. Handling Special Instructions

Let's create a version of the chat interface that can handle special instructions for the model.

In [ ]:
class AdvancedChatBot(ChatBot):
    def __init__(self, llm, system_prompt=None):
        super().__init__(llm, system_prompt)
        self.special_instructions = {}
    
    def add_instruction(self, key, instruction):
        """Add a special instruction to the chatbot."""
        self.special_instructions[key] = instruction
        print(f"Instruction '{key}' added.")
    
    def remove_instruction(self, key):
        """Remove a special instruction from the chatbot."""
        if key in self.special_instructions:
            del self.special_instructions[key]
            print(f"Instruction '{key}' removed.")
        else:
            print(f"Instruction '{key}' not found.")
    
    def chat(self, user_message, max_tokens=512, temperature=0.7, instruction_key=None):
        """Process a user message with optional special instruction."""
        # Use specific instruction if provided
        temp_system_prompt = None
        if instruction_key and instruction_key in self.special_instructions:
            # Save the original system prompt
            original_system_prompt = self.conversation_history[0]["content"]
            
            # Update with the special instruction
            temp_system_prompt = self.special_instructions[instruction_key]
            self.conversation_history[0]["content"] = temp_system_prompt
        
        # Call the parent chat method
        response = super().chat(user_message, max_tokens, temperature)
        
        # Restore the original system prompt if it was changed
        if temp_system_prompt is not None:
            self.conversation_history[0]["content"] = original_system_prompt
        
        return response

# Create an advanced chatbot
advanced_chatbot = AdvancedChatBot(llm)

# Add some special instructions
advanced_chatbot.add_instruction("poet", "Jesteś poetą. Odpowiadasz w formie wiersza, używając metafor i barwnego języka.")
advanced_chatbot.add_instruction("historian", "Jesteś historykiem. Odpowiadasz w szczegółowy sposób, podając daty, fakty i kontekst historyczny.")
advanced_chatbot.add_instruction("concise", "Odpowiadasz bardzo zwięźle, używając maksymalnie 3 zdań.")

# Test with different instructions
questions = [
    "Opowiedz mi o Warszawie.",
    "Czym jest szczęście?",
    "Jakie są najpopularniejsze sporty w Polsce?"
]

for question in questions:
    print(f"\n\n{'=' * 50}\nQuestion: {question}\n{'=' * 50}\n")
    
    print("[Standard response]")
    response = advanced_chatbot.chat(question)
    print(response)
    
    print("\n[Poet response]")
    response = advanced_chatbot.chat(question, instruction_key="poet")
    print(response)
    
    print("\n[Concise response]")
    response = advanced_chatbot.chat(question, instruction_key="concise")
    print(response)

## 9. Cleanup

When you're done with the model, it's a good practice to clean up resources.

In [ ]:
# Free the model from memory
del llm
del chatbot
del advanced_chatbot

# Force Python's garbage collector to run
import gc
gc.collect()

print("Model resources released.")

In this notebook, we've demonstrated how to implement a chat completion interface with the PLLuM-8x7B-chat model in GGUF format. We've shown:

1. How to create a basic chatbot with conversation history
2. How to format prompts for chat completion
3. How to use system prompts to guide the model's behavior
4. How to implement special instructions for different response styles
5. How to create an interactive chat interface

These techniques can be extended and customized for various applications, such as customer service chatbots, language learning assistants, or specialized knowledge agents.